In [1]:
# !pip install "tensorflow-text"
# !pip install einops

In [2]:
#EXP
# import pandas as pd
# convert csv to txt
def csv_to_txt( path = '/content/captions_csv.csv'):
  import pandas as pd
  data = pd.read_csv(path)
  data.dropna()
  print(data.head())
  column_contents = data['Caption'].astype(str).values.tolist()
  column_contents
  # Write the column contents to a text file
  with open('captions.txt', 'w', encoding='utf-8') as f:
    for item in column_contents:
        item = item.encode('utf-8').decode('unicode_escape')
        f.write("%s\n" % item)
# csv_to_txt()

In [3]:
#CONSTANTS
path_to_data_file = './bookstxt.txt'
DATA_LIMIT = 40
model_root_path = '../models/attention'
model_name = 'fc1'
model_path = f'{model_root_path}/{DATA_LIMIT}_model_{model_name}'
vectorizer_config_path = '../models/attention/text_processor_config.pkl'
vectorizer_weights_path = '../models/attention/text_processor_weights.pkl'

In [4]:
import numpy as np

import typing
from typing import Any, Tuple

import einops
# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker

import tensorflow as tf
import tensorflow_text as tf_text
import pathlib
import pickle

In [5]:
# path_to_zip = tf.keras.utils.get_file('spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip', extract=True)


In [6]:
path_to_file = pathlib.Path(path_to_data_file)
# path_to_file = pathlib.Path(path_to_zip).parent/'spa-eng/spa.txt'
def load_data(path):
  text = path.read_text(encoding='utf-8')
  lines = text.splitlines()
  print(type(lines))
  pairs = [line.split('\t') for line in lines]
  pairs = pairs[:DATA_LIMIT]
  # text = np.array(pairs)

  text = np.array([target for target in pairs])

  return text

data = load_data(path_to_file)
print(data.shape)

data = data[:, 0]
print(data.shape)
print(data)

<class 'list'>
(40, 1)
(40,)
['Charles Bukowski was an alcoholic, a womanizer, a chronic gambler, a lout, a cheapskate, a deadbeat, and on his worst days, a poet. He’s probably the last person on earth you would ever look to for life advice or expect to see in any sort of self-help book. Which is why he’s the perfect place to start. Bukowski wanted to be a writer. But for decades his work was rejected by almost every magazine, newspaper, journal, agent, and publisher he submitted to. His work was horrible, they said. Crude. Disgusting. Depraved. And as the stacks of rejection slips piled up, the weight of his failures pushed him deep into an alcohol-fueled depression that would follow him for most of his life. Bukowski had a day job as a letter-filer at a post office. He got paid shit money and spent most of it on booze. He gambled away the rest at the racetrack. At night, he would drink alone and sometimes hammer out poetry on his beat-up old typewriter. Often, he’d wake up on the flo

In [7]:
len(data)

40

In [8]:
BUFFER_SIZE = len(data)
BATCH_SIZE = 8
is_train = np.random.uniform(size=(len(data),)) < 0.9
train_raw = tf.data.Dataset.from_tensor_slices(data[is_train]).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
val_raw = tf.data.Dataset.from_tensor_slices(data[~is_train] ).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [9]:
len(train_raw)

5

In [10]:
for context_string in train_raw.take(1):
  print(context_string[:])
  break

tf.Tensor(
[b'About twenty-five hundred years ago, in the Himalayan foothills of present-day Nepal, there lived in a great palace a king who was going to have a son. For this son the king had a particularly grand idea: he would make the child\xe2\x80\x99s life perfect. The child would never know a moment of suffering\xe2\x80\x94every need, every desire, would be accounted for at all times. The king built high walls around the palace that prevented the prince from knowing the outside world. He spoiled the child, lavishing him with food and gifts, surrounding him with servants who catered to his every whim. And just as planned, the child grew up ignorant of the routine cruelties of human existence. All of the prince\xe2\x80\x99s childhood went on like this. But despite the endless luxury and opulence, the prince became kind of a pissed-off young man. Soon, every experience felt empty and valueless. The problem was that no matter what his father gave him, it never seemed enough, never mea

In [11]:
example_text = tf.constant("You have been invited to think of the two systems as agents within the mind.")
print(example_text.numpy())
print(tf_text.normalize_utf8(example_text, 'NFKD').numpy())

b'You have been invited to think of the two systems as agents within the mind.'
b'You have been invited to think of the two systems as agents within the mind.'


In [12]:
# @tf.keras.utils.register_keras_serializable(package='Custom', name=None)
def tf_lower_and_split_punct(text):
  # Split accented characters
  # text = tf_text.normalize_utf8(text, 'NFKD')
  text = tf.strings.lower(text)
  # Keep space, a to z, and select punctuation.
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
  # add space arround punctuation
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  # remove non-desplayable characters
  text = tf.strings.regex_replace(text, '[^\x00-\x7F]+', '')
  #Strip white space
  text = tf.strings.strip(text)

  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text


In [13]:
print(example_text.numpy().decode())
print(tf_lower_and_split_punct(example_text).numpy().decode())

You have been invited to think of the two systems as agents within the mind.
[START] you have been invited to think of the two systems as agents within the mind . [END]


In [14]:
#Text Vectorization for the context text data (spainish)
max_vocab_size = 30000
context_text_processor = tf.keras.layers.TextVectorization(standardize=tf_lower_and_split_punct, max_tokens=max_vocab_size, ragged=True)

In [15]:
context_text_processor.adapt(train_raw)
print(context_text_processor.get_vocabulary()[:20])


['', '[UNK]', '.', ',', 'the', 'and', 'to', 'a', 'of', 'you', 'that', 'i', 'in', 'was', 'it', 'on', 'his', 'he', 'for', 'with']


In [16]:
# pk.dump(context_text_processor.get_config(), open('text_processor_config.pkl', 'wb'))

In [17]:
# now layers can convert batch of strings to batch of token ids
example_tokens = context_text_processor(context_string)
print(example_tokens[:3, :])

<tf.RaggedTensor [[26, 22, 737, 540, 97, 251, 3, 12, 4, 1318, 1397, 8, 1049, 1143, 3, 68,
  519, 12, 7, 79, 296, 7, 320, 60, 13, 80, 6, 39, 7, 434, 2, 18, 28, 434, 4,
  320, 32, 7, 294, 1351, 137, 17, 55, 101, 4, 1637, 42, 216, 2, 4, 197, 55,
  133, 85, 7, 504, 8, 818, 219, 3, 117, 1533, 3, 55, 37, 1796, 18, 23, 34,
  265, 2, 4, 320, 1666, 543, 697, 171, 4, 296, 10, 1047, 4, 290, 104, 525,
  4, 297, 150, 2, 17, 865, 4, 197, 3, 1242, 43, 19, 341, 5, 1364, 3, 806,
  43, 19, 927, 60, 1652, 6, 16, 117, 678, 2, 5, 58, 21, 482, 3, 4, 197, 552,
  47, 1300, 8, 4, 959, 1570, 8, 326, 1445, 2, 34, 8, 4, 1046, 1639, 383, 15,
  50, 28, 2, 29, 595, 4, 1468, 1203, 5, 1108, 3, 4, 290, 1709, 229, 8, 7,
  1070, 376, 311, 2, 433, 3, 117, 344, 191, 582, 5, 710, 2, 4, 130, 13, 10,
  57, 309, 24, 16, 116, 335, 43, 3, 14, 133, 283, 346, 3, 133, 1186, 144, 2,
  38, 1245, 91, 184, 3, 4, 290, 885, 49, 8, 4, 296, 2, 2, 2, 27]            ,
 [26, 97, 251, 3, 40, 11, 13, 92, 375, 5, 831, 3, 11, 654, 7, 361, 291, 3,

In [18]:
#The get_vocabulary method can be used to convert token IDs back to text:
context_vocab = np.array(context_text_processor.get_vocabulary())
tokens = context_vocab[example_tokens[0].numpy()]
tokens = ' '.join(tokens)
print(tokens)

[START] about twentyfive hundred years ago , in the himalayan foothills of presentday nepal , there lived in a great palace a king who was going to have a son . for this son the king had a particularly grand idea he would make the childs life perfect . the child would never know a moment of sufferingevery need , every desire , would be accounted for at all times . the king built high walls around the palace that prevented the prince from knowing the outside world . he spoiled the child , lavishing him with food and gifts , surrounding him with servants who catered to his every whim . and just as planned , the child grew up ignorant of the routine cruelties of human existence . all of the princes childhood went on like this . but despite the endless luxury and opulence , the prince became kind of a pissedoff young man . soon , every experience felt empty and valueless . the problem was that no matter what his father gave him , it never seemed enough , never meant anything . so late one 

In [19]:

def process_text(context):
  target = context_text_processor(context)
  context = context_text_processor(context).to_tensor()
  # print(type(context))
  # targ_in = target[:, :-1].to_tensor() #take everthing in axiz = 0 and take everything except the last in axis = 2
  # targ_in = target[:, 1:].to_tensor()
  # targ_out = target[:, :-1].to_tensor()
  target = target[:, 1:]
  targ_in = target[:, :-1].to_tensor() #take everthing in axiz = 0 and take everything except the last in axis = 2
  targ_out = target[:, 1:].to_tensor()

  

  print('process_text')

  return (context, targ_in), targ_out

train_ds = train_raw.map(process_text, tf.data.AUTOTUNE)
val_ds = val_raw.map(process_text, tf.data.AUTOTUNE)

process_text
process_text


In [20]:
len(train_ds)


5

In [21]:
for (example_context_tokens, target_in_tokens), target_out_tokens in train_ds.take(2):

  print(example_context_tokens[0, :10].numpy(), example_context_tokens[0, :].numpy().shape)
  print(target_in_tokens[0, :10].numpy(), target_in_tokens[0, :].numpy().shape)
  print(target_out_tokens[0, :10].numpy(), target_out_tokens[0, :].numpy().shape)
  print()

[ 26  36  11 170   9   3  24  54   9  89] (225,)
[ 36  11 170   9   3  24  54   9  89  49] (223,)
[ 11 170   9   3  24  54   9  89  49   8] (223,)

[ 26  36  11 142 530   7 815   3  11  55] (215,)
[ 36  11 142 530   7 815   3  11  55 530] (213,)
[ 11 142 530   7 815   3  11  55 530  91] (213,)



In [22]:
# target_in_tokens = target_in_tokens + 1
# print(target_in_tokens[0, :10].numpy(),'\n', target_out_tokens[0, :10].numpy())

In [23]:
'''
The encoder:
  1. Takes a list of token IDs (from context_text_processor).
  2. Looks up an embedding vector for each token (Using a layers.Embedding).
  3. Processes the embeddings into a new sequence (Using a bidirectional layers.GRU).
  4. Returns the processed sequence. This will be passed to the attention head.
'''


class Encoder(tf.keras.layers.Layer):
  def __init__(self, text_processor, units):
    super(Encoder, self).__init__()
    self.text_processor = text_processor
    self.vocab_size = text_processor.vocabulary_size()
    self.units = units

    #The embedding layer converts tokens into vectors
    self.embedding = tf.keras.layers.Embedding(self.vocab_size, self.units, mask_zero=True)

    #The RNN layer processes those vectors sequentially
    self.rnn = tf.keras.layers.Bidirectional(merge_mode='sum', 
                                             layer=tf.keras.layers.GRU(self.units, return_sequences=True, recurrent_initializer='glorot_uniform' ))
  
  def call(self, x):
    x = self.embedding(x)
    x = self.rnn(x)
    # print('call')
    return x

  def convert_input(self, texts):
    texts = tf.convert_to_tensor(texts)
    if len(texts.shape) == 0:
      texts = tf.convert_to_tensor(texts)[tf.newaxis]
    context = self.text_processor(texts).to_tensor()
    context = self(context)
    return context


In [24]:
UNITS = 256

In [25]:
# Encode the input sequence.
encoder = Encoder(context_text_processor, UNITS)
ex_context = encoder(example_context_tokens)

print(f'Context tokens, shape (batch, s): {example_context_tokens.shape}')
print(f'Encoder output, shape (batch, s, units): {ex_context.shape}')

Context tokens, shape (batch, s): (8, 215)
Encoder output, shape (batch, s, units): (8, 215, 256)


In [26]:
class CrossAttention(tf.keras.layers.Layer):
  def __init__(self, units, **kwargs):
    super().__init__()

    self.mha = tf.keras.layers.MultiHeadAttention(key_dim=units, num_heads=1, **kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

  def call(self, x, context):
    atten_output, atten_score = self.mha(query=x, value=context, return_attention_scores=True)
    x = self.add([x, atten_output])
    x = self.layernorm(x)
    return x

In [27]:
attention_layer = CrossAttention(UNITS)
# Attend to the encoded tokens
embed = tf.keras.layers.Embedding(context_text_processor.vocabulary_size(),
                                  output_dim=UNITS, mask_zero=True)


In [28]:
'''
The decoder's job is to generate predictions for the next token at each location in the target sequence.
  1. It looks up embeddings for each token in the target sequence.
  2. It uses an RNN to process the target sequence, and keep track of what it has generated so far.
  3. It uses RNN output as the "query" to the attention layer, when attending to the encoder's output.
  4. At each location in the output it predicts the next token.
'''

class Decoder(tf.keras.layers.Layer):
  @classmethod
  def add_method(cls, fun):
    setattr(cls, fun.__name__, fun)
    return fun

  def __init__(self, text_processor, units):
    super(Decoder, self).__init__()
    self.text_processor = text_processor
    self.vocab_size = text_processor.vocabulary_size()
    self.word_to_id = tf.keras.layers.StringLookup(vocabulary=text_processor.get_vocabulary(), mask_token='', oov_token='[UNK]')
    self.id_to_word = tf.keras.layers.StringLookup(vocabulary=text_processor.get_vocabulary(), mask_token='', oov_token='[UNK]', invert=True)
    self.start_token = self.word_to_id('[START]')
    self.end_token = self.word_to_id('[END]')

    self.units = units

    # 1. The embedding layer converts token IDs to vectors
    self.embedding = tf.keras.layers.Embedding(self.vocab_size, self.units, mask_zero=True)
    # 2. The RNN keeps track of what's been generated so far.
    self.rnn = tf.keras.layers.GRU(self.units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
    #3. The RNN output will be the query for the attention layer.
    self.attention = CrossAttention(self.units)

    self.fc1 = tf.keras.layers.Dense(self.units, activation='relu')
    # 4. This fully connected layer produces the logits for each output token.
    self.output_layer = tf.keras.layers.Dense(self.vocab_size)

  def call(self, context, x, state=None, return_state=False):
    #Lookup for embeddings
    x = self.embedding(x)
    #Process the target sequence
    x, state = self.rnn(x, initial_state=state)
    #Use the rnn output as the query for the attention over the context
    x = self.attention(x, context)

    x = self.fc1(x)

    #generate logit predictyions for the next token
    logits = self.output_layer(x)


    if return_state:
      return logits, state
    return logits




In [29]:
decoder = Decoder(context_text_processor, UNITS)

In [30]:
@Decoder.add_method
def get_initial_state(self, context):
  batch_size = tf.shape(context)[0]
  print(batch_size)
  start_tokens = tf.fill([batch_size, 1], self.start_token)

  # print(start_tokens, 'satrt token')
  # print(tf.constant(context), 'context')
  done = tf.zeros([batch_size, 1], dtype=tf.bool)
  embedded = self.embedding(start_tokens)
  return start_tokens, done, self.rnn.get_initial_state(embedded)[0]

# print(example_context_tokens, 'ex_context_tokens')
# ex_context = encoder(example_context_tokens)
# next_token, done, state = decoder.get_initial_state(ex_context)
# print(next_token, state)

In [31]:
@Decoder.add_method
def tokens_to_text(self, tokens):
  words = self.id_to_word(tokens)
  result = tf.strings.reduce_join(words, axis=-1, separator=' ')
  result = tf.strings.regex_replace(result, '^ *\[START\] *', '')
  result = tf.strings.regex_replace(result, ' *\[END\] *$', '')
  return result

# decoder.tokens_to_text(example_context_tokens)

In [32]:
@Decoder.add_method
def get_next_token(self, context, next_token, done, state, temperature = 0.0):

  logits, state = self(context, next_token, state = state, return_state=True) 

  if temperature == 0.0:
    next_token = tf.argmax(logits, axis=-1)
  else:
    logits = logits[:, -1, :]/temperature
    next_token = tf.random.categorical(logits, num_samples=1)

  # If a sequence produces an `end_token`, set it `done`
  # done = done | (next_token == self.end_token)
  # # Once a sequence is done it only produces 0-padding.
  # next_token = tf.where(done, tf.constant(0, dtype=tf.int64), next_token)

  return next_token, done, state

In [33]:
# Setup the loop variables.
next_token, done, state = decoder.get_initial_state(ex_context)
tokens = []

for n in range(10):
  # Run one step.
  next_token, done, state = decoder.get_next_token(
      ex_context, next_token, done, state, temperature=1.0)
  # Add the token to the output.
  tokens.append(next_token)

# Stack all the tokens together.
tokens = tf.concat(tokens, axis=-1) # (batch, t)

# Convert the tokens back to a a string
result = decoder.tokens_to_text(tokens)
result[:3].numpy()

tf.Tensor(8, shape=(), dtype=int32)


array([b'catch experimenting wednesday point faint author morning gotten flung age',
       b'batty membership bleeding ability quite visited rape regularly wait humdrum',
       b'rejecting fill young alcoholic wonky , hell immediate place studied'],
      dtype=object)

In [34]:
class TextGen(tf.keras.Model):
  @classmethod
  def add_method(cls, fun):
    setattr(cls, fun.__name__, fun)
    return fun

  def __init__(self, units, context_text_processor, target_text_processor):
    super().__init__()
    #Build the encoder and the decoder
    self.encoder = Encoder(context_text_processor, units)
    self.decoder = Decoder(context_text_processor, units)

  def call(self, inputs):
    # print("here1")
    context, x = inputs
    # print("here2")

    context = self.encoder(context)
    # print("here3")

    logits = self.decoder(context, x)
    # print("here4")

    try:
      del logits._keras_mask
    except AttributeError:
      pass
    # print("here5", logits)
    return logits
  
  def gen(self,
                texts, *,
                max_length=50,
                temperature=0.0):
    # Process the input texts
    context = self.encoder.convert_input(texts)
    batch_size = tf.shape(texts)[0]

    # Setup the loop inputs
    tokens = []
    attention_weights = []
    next_token, done, state = self.decoder.get_initial_state(context)

    for _ in range(max_length):
      # Generate the next token
      next_token, done, state = self.decoder.get_next_token(context, next_token, done,  state, temperature)

      # Collect the generated tokens
      tokens.append(next_token)
      # attention_weights.append(self.decoder.last_attention_weights)

      # if tf.executing_eagerly() and tf.reduce_all(done):
      #   break

    # Stack the lists of tokens and attention weights.
    tokens = tf.concat(tokens, axis=-1)   # t*[(batch 1)] -> (batch, t)
    # self.last_attention_weights = tf.concat(attention_weights, axis=1)  # t*[(batch 1 s)] -> (batch, t s)

    result = self.decoder.tokens_to_text(tokens)
    return result

    

In [35]:
model = TextGen(UNITS, context_text_processor, context_text_processor)

In [36]:
def masked_loss(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    # print(y_true.shape, y_pred.shape, "masked_loss")
    loss = loss_fn(y_true, y_pred)
    

    # Mask off the losses on padding.
    mask = tf.cast(y_true != 0, loss.dtype)
    loss *= mask

    # Return the total.
    return tf.reduce_sum(loss)/tf.reduce_sum(mask)

In [37]:
#custom accuracy
def masked_acc(y_true, y_pred):
  # Calculate the loss for each item in the batch
  y_pred = tf.argmax(y_pred, axis=-1)
  y_pred = tf.cast(y_pred, y_true.dtype)

  match = tf.cast(y_true == y_pred, dtype=tf.float32)
  mask = tf.cast(y_true != 0, tf.float32)

  return tf.reduce_sum(match) / tf.reduce_sum(mask)


In [38]:
model.compile(optimizer='adam', loss=masked_loss, metrics=[masked_acc, masked_loss])

In [39]:
vocab_size = 1.0 * context_text_processor.vocabulary_size()

{"expected_loss": tf.math.log(vocab_size).numpy(),
 "expected_acc": 1/vocab_size}

{'expected_loss': 7.4977617, 'expected_acc': 0.0005543237250554324}

In [40]:
# model.evaluate(val_ds, steps=20, return_dict=True)

In [41]:
checkpoint_path = model_root_path + f"/{DATA_LIMIT}_cp_{model_name}.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, verbose=1)

In [42]:
history = model.fit(
    train_ds.repeat(), 
    epochs=10,
    steps_per_epoch = 20,
    validation_data=val_ds,
    validation_steps = 20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=5, monitor='masked_acc')])

Epoch 1/10
20/20 [==============================] - 48s 2s/step - loss: 6.3695 - masked_acc: 0.0640 - masked_loss: 6.3264 - val_loss: 6.3365 - val_masked_acc: 0.0532 - val_masked_loss: 6.3365
Epoch 2/10
20/20 [==============================] - 25s 1s/step - loss: 5.5661 - masked_acc: 0.1394 - masked_loss: 5.5444
Epoch 3/10
20/20 [==============================] - 24s 1s/step - loss: 4.4652 - masked_acc: 0.2332 - masked_loss: 4.4551
Epoch 4/10
20/20 [==============================] - 23s 1s/step - loss: 3.1569 - masked_acc: 0.3695 - masked_loss: 3.1568
Epoch 5/10
20/20 [==============================] - 22s 1s/step - loss: 1.7840 - masked_acc: 0.6292 - masked_loss: 1.7928
Epoch 6/10
20/20 [==============================] - 23s 1s/step - loss: 0.8442 - masked_acc: 0.8528 - masked_loss: 0.8491
Epoch 7/10
20/20 [==============================] - 22s 1s/step - loss: 0.3702 - masked_acc: 0.9525 - masked_loss: 0.3720
Epoch 8/10
20/20 [==============================] - 23s 1s/step - loss: 0.17

In [43]:
result = model.gen(['This was one of my favorite'], max_length=100)
result = result[0].numpy().decode()
print(result)

tf.Tensor(1, shape=(), dtype=int32)
i was doing , but i figured if i was going to be broke and miserable , i might as well be while working on my own terms . and at that time , all i seemed to really care about was chasing girls . so fuck it , i decided to start a blog about my crazy dating life . that first morning that i woke up selfemployed , terror quickly consumed me . i found myself sitting with my laptop and realized , for the first time , that i was entirely responsible for all of my own


### SAVE 

In [44]:
# model_path = f'../models/attention/{DATA_LIMIT}_model.tf'
# vectorizer_config_path = '../models/attention/text_processor_config.pkl'
# vectorizer_weights_path = '../models/attention/text_processor_weights.pkl'


In [45]:
# #save the vectorizaation layer
# pickle.dump(context_text_processor.get_config(), open(vectorizer_config_path, 'wb'))
# weights = context_text_processor.get_weights()
# text_processor_weights = pickle.dump(weights, open(vectorizer_weights_path, "wb"))
# #save the final fully trained  model
# model.save(model_path)         

In [46]:
# loaded_model = None
# loaded_model = tf.keras.models.load_model(model_path, custom_objects={'tf_lower_and_split_punct': tf_lower_and_split_punct}, compile=False)

In [47]:
# result = loaded_model.predict(['This was one of my favorite'])
# result = result[0].numpy().decode()
# print(result)

In [48]:
# import pickle

# weights = context_text_processor.get_weights()

# text_processor_weights = pickle.dump(weights, open("text_processor_weights.pkl", "wb"))


# loaded_config = pickle.load(open("text_processor_config.pkl", 'rb'))
# loaded_weights = pickle.load(open("text_processor_weights.pkl", 'rb'))
# text_processor = tf.keras.layers.TextVectorization.from_config(loaded_config)
# text_processor.set_weights(loaded_weights)

# new_model = TextGen(UNITS, text_processor, text_processor)

In [49]:
# new_model.load_weights(model_path)

In [50]:
# result = new_model.gen(['Hello world'])
# result = result[0].numpy().decode()
# print(result)

In [51]:
# def create_model(path_to_model, path_to_vectorizer_config, path_to_vectorizer_weights):
#     loaded_config = pickle.load(open(path_to_vectorizer_config, 'rb'))
#     loaded_weights = pickle.load(open(path_to_vectorizer_weights, 'rb'))

#     text_processor = tf.keras.layers.TextVectorization.from_config(loaded_config)
#     text_processor.set_weights(loaded_weights)

#     new_model = TextGen(UNITS, text_processor, text_processor)
#     new_model.load_weights(path_to_model)

#     return new_model




In [52]:
# textGenModel = create_model(path_to_model=model_path, path_to_vectorizer_config='text_processor_config.pkl', path_to_vectorizer_weights='text_processor_weights.pkl')

In [53]:
# textGenModel.gen(['This was one of my favorite and'])

In [44]:
class Export(tf.Module):
    def __init__(self, model):
        self.model = model
    @tf.function(input_signature=[tf.TensorSpec(dtype=tf.string, shape=[None])])
    def generate(self, inputs):
        return self.model.gen(texts=inputs, max_length=500)

export = Export(model)
# _ = export.generate(tf.constant(["Hello World"]))
tf.saved_model.save(export, model_path, signatures={'serving_defualt': export.generate})

Tensor("strided_slice_1:0", shape=(), dtype=int32)


In [ ]:
reloaded = tf.saved_model.load(model_path)

_ = reloaded.generate(tf.constant(['Hello World']))  #warmup
result = reloaded.generate(tf.constant(['Blog post']))
result[0].numpy().decode()

In [ ]:
result = reloaded.generate(tf.constant(['Fuck']), max_length=100)
result[0].numpy().decode()